In [1]:
import json
import pandas as pd

In [2]:
def file_path (model):
    filePath=f"/Volumes/Jennie/Reasoning/FinReasoning/output/{model}_output.json"
    return filePath

In [3]:
def evaluate(eva_file_path):
    data=pd.read_json(eva_file_path)
    # a=0
    # for question in data:
    #     if question["Model Answer"] == question["Answer"]:
    #         a+=1
    #     else:
    #         a+=0
    data["isCorrect"] = data.apply(lambda row: 1 if row['Model Answer'] == row['Answer'] else 0, axis=1)
    a=data["isCorrect"].sum()
    b=data['isCorrect'].count()
    accuracy=(a/b*100).__round__(2)
    return accuracy

In [32]:
def evaluate_topics(eva_file_path):
    data=pd.read_json(eva_file_path)
    # 计算每个答案是否正确
    data["isCorrect"] = data.apply(lambda row: 1 if row['Model Answer'] == row['Answer'] else 0, axis=1)

    data['General Topics']=data['General Topics'].apply(lambda x: 'Others' if x in other_topics  else x)

    # 计算每个 General Topics 的正确答案数量
    correct_by_topic = data.groupby('General Topics')['isCorrect'].sum().reset_index()

    # 计算每个 General Topics 的总答案数量
    count_by_topic = data.groupby('General Topics')['isCorrect'].count().reset_index()

    # 合并两个 DataFrame
    accuracy_by_topic = pd.merge(correct_by_topic, count_by_topic, on='General Topics')

    # 计算正确率
    accuracy_by_topic['accuracy%'] = round(accuracy_by_topic['isCorrect_x'] / accuracy_by_topic['isCorrect_y'] * 100, 2)

    # 重命名列
    accuracy_by_topic.columns = ['General Topics', 'correct', 'total', 'accuracy%']

    # 计算总体正确率
    total_correct = data['isCorrect'].sum()
    total_count = data['isCorrect'].count()
    overall_accuracy = round((total_correct / total_count) * 100, 2)

    # 创建一个新的 DataFrame 来存储总体正确率
    overall_accuracy_df = pd.DataFrame([['Overall', total_correct, total_count, overall_accuracy]], columns=['General Topics', 'correct', 'total', 'accuracy%'])

    # 将总体正确率 DataFrame 与 accuracy_by_topic 进行合并
    final_df = pd.concat([accuracy_by_topic, overall_accuracy_df], ignore_index=True)

    # 打印结果
    accuracy=list(final_df['accuracy%'])
    return accuracy
    

In [48]:
models=[
        "gpt_o1","gpt_o1_rag",
        "Gemini_Pro1","Gemini_Pro1_rag",
        "deepseek","deepseek_rag",
        "llama","llama_rag",
        "gpt_4o","gpt_4o_rag",
        "gemini_1.5_pro","gemini_1.5_pro_rag",
        "qwen","qwen_rag",
        "Llava","Llava_rag",
        "Llama_3.2_vision","Llama_3.2_vision_rag",
        "qwen2_vl","qwen2_vl_rag",
        ]
topics=['Alternative Investments',
#  'Asset Allocation',
#  'Capital Market Expectations',
 'Corporate Finance',
 'Credit Risk Measurement and Management',
 'Derivatives',
#  'Derivatives and Currency Management',
 'Economics',
 'Equity Investments',
#  'Equity Portfolio Management',
#  'Ethicaland Professional Standards',
 'Financial Markets and Products',
 'Financial Reporting and Analysis',
 'Fixed Income',
#  'Fixed-Income Portfolio Management',
 'Foundation of Risk Management',
 'Liquidity and Treasury Risk Measurement',
 'Market Risk Measurement and Management',
 'Operational Risk',
#  'Operational Risk and Resiliency',
 'Portfolio Management',
 'Quantitative Methods',
 'Risk Management and Investment Management',
 'Valuation and Risk Models',
 'Overall']
other_topics=['Fixed-Income Portfolio Management','Operational Risk','Equity Portfolio Management','Derivatives and Currency Management',
              'Asset Allocation','Ethicaland Professional Standards','Capital Market Expectations']
column=['Alternative Investments',
#  'Asset Allocation',
#  'Capital Market Expectations',
 'Corporate Finance',
 'Credit Risk Measurement and Management',
 'Derivatives',
#  'Derivatives and Currency Management',
 'Economics',
 'Equity Investments',
#  'Equity Portfolio Management',
#  'Ethicaland Professional Standards',
 'Financial Markets and Products',
 'Financial Reporting and Analysis',
 'Fixed Income',
#  'Fixed-Income Portfolio Management',
 'Foundation of Risk Management',
 'Liquidity and Treasury Risk Measurement',
 'Market Risk Measurement and Management',
 'Operational Risk',
#  'Operational Risk and Resiliency',
 'Portfolio Management',
 'Quantitative Methods',
 'Risk Management and Investment Management',
 'Valuation and Risk Models',
 "Others",
 'Overall']

In [49]:
modelAcc=[]
for model in models:
    modelAcc.append(evaluate(file_path(model)))
result={
    'Model': models,
    'Accuracy%': modelAcc
}

result=pd.DataFrame(result)
result

,Model,Accuracy%
0,gpt_o1,67.67
1,gpt_o1_rag,80.41
2,Gemini_Pro1,70.39
3,Gemini_Pro1_rag,78.68
4,deepseek,70.90
5,deepseek_rag,78.75
6,llama,35.20
7,llama_rag,47.95
8,gpt_4o,79.69
9,gpt_4o_rag,85.28


In [11]:
result.to_csv('/Volumes/Jennie/Reasoning/FinReasoning/evaluation/Model_Accuracy.csv',index=False)

In [47]:
result.sort_values(by='Accuracy%', ascending=False)

,Model,Accuracy%
11,gemini_1.5_pro_rag,86.00
9,gpt_4o_rag,85.28
1,gpt_o1_rag,80.41
8,gpt_4o,79.69
5,deepseek_rag,78.75
3,Gemini_Pro1_rag,78.68
10,gemini_1.5_pro,77.53
13,qwen_rag,76.68
4,deepseek,70.90
2,Gemini_Pro1,70.39


In [50]:
topic_Acc=[]
for model in models:
    topic_Acc.append(evaluate_topics(file_path(model)))
topic_result=pd.DataFrame(topic_Acc,index=models, columns=column)
topic_result

,Alternative Investments,Corporate Finance,Credit Risk Measurement and Management,Derivatives,Economics,Equity Investments,Financial Markets and Products,Financial Reporting and Analysis,Fixed Income,Foundation of Risk Management,Liquidity and Treasury Risk Measurement,Market Risk Measurement and Management,Operational Risk,Portfolio Management,Quantitative Methods,Risk Management and Investment Management,Valuation and Risk Models,Others,Overall
gpt_o1,67.50,67.71,71.07,65.98,74.68,63.58,74.16,67.07,78.49,63.23,61.76,63.72,57.14,51.10,65.79,77.26,55.66,67.13,67.67
gpt_o1_rag,77.50,83.33,81.76,79.38,85.99,77.78,87.64,77.33,88.71,75.97,76.24,74.34,70.33,65.38,80.42,86.92,70.75,84.08,80.41
Gemini_Pro1,76.25,65.62,49.37,80.41,81.65,76.54,76.32,71.43,59.12,66.45,67.65,64.60,71.43,58.01,76.84,81.62,60.38,68.51,70.39
Gemini_Pro1_rag,81.25,78.12,62.42,83.51,84.81,84.57,84.91,78.78,69.36,72.90,80.39,72.57,81.32,70.98,83.68,85.67,70.75,79.58,78.68
deepseek,73.75,69.79,58.49,73.20,80.38,76.54,72.66,73.17,77.96,69.68,53.92,55.75,64.84,54.40,78.95,82.87,63.21,69.20,70.90
deepseek_rag,80.00,80.21,65.41,82.47,88.61,82.10,81.65,80.08,83.33,74.19,70.59,63.72,71.43,63.89,85.79,88.16,73.58,79.58,78.75
llama,36.25,37.50,49.69,26.80,43.04,38.89,25.09,35.37,67.20,32.90,24.51,19.47,37.36,26.92,31.58,40.19,26.42,26.99,35.20
llama_rag,44.87,46.88,59.87,41.24,58.86,49.38,40.47,46.34,77.96,48.37,42.00,35.40,50.55,35.71,47.37,52.34,40.00,38.49,47.95
gpt_4o,76.25,75.79,72.90,85.57,89.03,86.42,81.58,85.31,88.52,84.40,74.49,63.39,69.66,63.22,85.26,85.05,70.87,75.00,79.69
gpt_4o_rag,88.75,87.37,81.29,89.69,90.32,88.89,87.97,89.80,91.80,88.73,82.65,71.43,73.03,74.14,87.89,87.54,80.58,81.60,85.28


In [51]:
topic_result.to_csv('/Volumes/Jennie/Reasoning/FinReasoning/evaluation/Topic_Accuracy1.csv')

In [5]:
data=pd.read_csv('/Users/sden118/Desktop/FinReasoning/evaluation/errorType.csv')
data.groupby(['Error']).count()

,ID,error type,Model
Error,,,
calculation,276,276,276
image,102,102,102
option,147,147,147
other,338,338,338
question,2315,2315,2315
